In [ ]:
import os
os.chdir('./..')

import csv
import pickle
import Methods.totals as totals
import Methods.analysis as analysis
import Methods.common as common
import pandas as pd

with open("sorted_dataset_bugfix.pkl", "rb") as f:
    dataset_dict = pickle.load(f)
    
import functools
import operator
import collections

In [ ]:
"""
    Getting totals
    Getting the final classification for every PR and calculating totals
"""
repos = 342
for repo_file in range(0, repos):
    variant1 = dataset_dict[repo_file]['source']
    variant2 = dataset_dict[repo_file]['destination']
    
    print(variant1, '-', variant2)
    
    try:
        all_results = common.readResults(repo_file, variant1)
        
        pr_class = totals.final_class(all_results[0])
        all_counts = totals.count_all_classifications(pr_class)

        with open('Repos_totals/'+ str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_totals.pkl', 'wb') as f:
            pickle.dump([pr_class, all_counts],f)
    except Exception as e:
        print(e)

In [ ]:
"""
    Plotting the distribution of classifications
"""

total_all = 0
total_mo_all = 0
total_ed_all = 0
total_sp_all = 0
total_na_all = 0

all_results = []

repos = 342
for repo_file in range(0,repos):
    try:
        variant1 = dataset_dict[repo_file]['source']
        variant2 = dataset_dict[repo_file]['destination']
        variant1_prs = dataset_dict[repo_file]['pr'].split('/')

        all_totals= totals.read_totals(repo_file, variant1)

        total_NA = 0
        total_ED = 0
        total_MO = 0
        total_CC = 0
        total_SP= 0
        total_NE = 0
        total_ERROR = 0

        for pr in all_totals[0]:
            verdict = all_totals[0][pr]['class']
            if verdict == 'ED':
                total_ED += 1
            elif verdict =='MO':
                total_MO += 1
            elif verdict == 'SP':
                total_SP += 1
            elif verdict == 'NA':
                total_NA += 1
            elif verdict == 'CC':
                total_CC += 1
            elif verdict =='NE':
                total_NE += 1
            elif verdict == 'ERROR':
                total_ERROR += 1
                
            total_mid = total_MO+ total_ED + total_SP
            total_all += total_mid
            
        total_total =len(variant1_prs)
        
        total_mo_all += total_MO
        total_ed_all += total_ED
        total_sp_all += total_SP
        total_na_all += total_NA

        totals_list = [total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR]

#         analysis.all_class_bar(totals_list, repo_file, variant1, variant2, False)
#         analysis.all_class_pie(totals_list, repo_file, variant1, variant2, False)

        all_results.append([variant1, variant2, repo_file, total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR])

    except Exception as e:
        print(variant1)
        print("Exception: ", e, '\n\n')

header = ["variant1", "variant2", "repo_file", "total_MO", "total_ED", "total_SP", "total_CC", "total_NE", "total_NA"," total_ERROR"]
with open("results.csv", "w") as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(all_results)

In [ ]:
print('MO = ',total_mo_all)
print('ED = ',total_ed_all)
print('SP = ',total_sp_all)
print('NI = ', total_na_all)

In [ ]:
47 + 54 + 8

# Analysis 2.0

## Analysis of the new data

In [ ]:
def merge_dict(dict1, dict2):
    return {**dict1, **dict2}

directory = 'Repos_results_bugfix'

totals_ = {}
repo_nr = 0

pair_set = set()

i = 0
prev_pair_id = '0'

all_pr_class_list = []
all_counts_list = []

for file in os.listdir(directory):
    pair_id = file.split('_')[0]
    pair_set.add(pair_id)
    if pair_id != prev_pair_id:
        dict_final ={}
        for i in all_pr_class_list:
            dict_final=merge_dict(dict_final, i)
        
        try:
            all_counts_all = dict(functools.reduce(operator.add,map(collections.Counter, all_counts_list)))
            totals_[pair_id] ={}
            totals_[pair_id]['pr_class'] = dict_final
            totals_[pair_id]['all_counts'] = all_counts_all
        except:
            print(pair_id)
            print(all_counts_list)
            totals_[pair_id] ={}
            totals_[pair_id]['pr_class'] = dict_final
            totals_[pair_id]['all_counts'] = []
            
        all_pr_class_list = []
        all_counts_list = []
        
        prev_pair_id = pair_id
    with open(f'{directory}/{file}', 'rb') as f:
        try:
            all_results = pickle.load(f)[0]

            pr_class = totals.final_class(all_results)
            all_pr_class_list.append(pr_class)
            all_counts = totals.count_all_classifications(pr_class)
            all_counts_list.append(all_counts)
        except EOFError:
            continue
#             with open('Repos_totals/'+ str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_totals.pkl', 'wb') as f:
#                 pickle.dump([pr_class, all_counts],f)
#             except Exception as e:
#                 print(e)

In [ ]:
"""
    Plotting the distribution of classifications
"""

total_all = 0
total_mo_all = 0
total_ed_all = 0
total_sp_all = 0
total_cc_all = 0
total_ne_all = 0
total_error_all = 0
total_na_all = 0

all_results = []

repos = 342
for repo_file in totals_:
    try:
        variant1 = dataset_dict[int(repo_file)]['source']
        variant2 = dataset_dict[int(repo_file)]['destination']
        variant1_prs = dataset_dict[int(repo_file)]['pr'].split('/')

        all_totals= totals_[repo_file]['pr_class']

        total_NA = 0
        total_ED = 0
        total_MO = 0
        total_CC = 0
        total_SP= 0
        total_NE = 0
        total_ERROR = 0

        for pr in all_totals:
            verdict = all_totals[pr]['class']
            if verdict == 'ED':
                total_ED += 1
            elif verdict =='MO':
                total_MO += 1
            elif verdict == 'SP':
                total_SP += 1
            elif verdict == 'NA':
                total_NA += 1
            elif verdict == 'CC':
                total_CC += 1
            elif verdict =='NE':
                total_NE += 1
            elif verdict == 'ERROR':
                total_ERROR += 1
                
            total_mid = total_MO+ total_ED + total_SP
            total_all += total_mid
            
        total_total =len(variant1_prs)
        
        total_mo_all += total_MO
        total_ed_all += total_ED
        total_sp_all += total_SP
        total_na_all += total_NA

        total_cc_all += total_CC
        total_ne_all += total_NE
        total_error_all += total_ERROR

        totals_list = [total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR]

#         analysis.all_class_bar(totals_list, repo_file, variant1, variant2, False)
#         analysis.all_class_pie(totals_list, repo_file, variant1, variant2, False)

        all_results.append([variant1, variant2, repo_file, total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR])

    except Exception as e:
        print(variant1)
        print("Exception: ", e, '\n\n')

# header = ["variant1", "variant2", "repo_file", "total_MO", "total_ED", "total_SP", "total_CC", "total_NE", "total_NA"," total_ERROR"]
# with open("results_16_05_22.csv", "w") as f:
#     writer = csv.writer(f)
#     # write the header
#     writer.writerow(header)

#     # write multiple rows
#     writer.writerows(all_results)

In [ ]:
print('MO = ',total_mo_all)
print('ED = ',total_ed_all)
print('SP = ',total_sp_all)
print('NI = ', total_na_all)

print('CC = ',total_cc_all)
print('NE = ',total_ne_all)
print('ERROR = ', total_error_all)

In [ ]:
import matplotlib.pyplot as plt

total_hist = [total_mo_all, total_ed_all, total_sp_all, total_na_all, total_cc_all, total_ne_all, total_error_all]

left = [1, 2, 3, 4, 5, 6, 7]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI', 'CC', 'NE', 'ERROR'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3", "#ff7f00", "#ffff33", "#a65628"])

# Percentage interesting

In [ ]:
total_classified = total_mo_all+total_ed_all+total_sp_all+total_na_all+total_cc_all+total_error_all+total_ne_all
print(f'total classified = {total_classified}')
print(f'total interesting = {100*(total_mo_all+total_ed_all+total_sp_all)/(total_classified-45)}%')
print(f'total unintersting = {100*total_na_all/total_classified}%')
print(f'total error = {100*total_error_all/total_classified}%')
print(f'total non existant = {100*total_ne_all/total_classified}%')
print(f'total cannot classify = {100*total_cc_all/total_classified}%')

# File classification distribution

In [ ]:
import Methods.classifier as classifier

classifications = {}

directory = 'Repos_results_bugfix'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in classifications:
                classifications[pair_id] = {}

            for i in results_file:
                classifications[pair_id][i] = {}

                try:
                    for file_exec in results_file[i]:
                        ext = file_exec.split('.')[-1]
                        classifications[pair_id][i][file_exec]={}
                        classifications[pair_id][i][file_exec]['ext'] = ext
                        classifications[pair_id][i][file_exec]['hunks'] = []
                        classifications[pair_id][i][file_exec]['class'] = ''
                        
                        patchClass = results_file[i][file_exec]['result']['patchClass']
                        
                        classifications[pair_id][i][file_exec]['class'] = patchClass
                        
                        if patchClass in ['MO', 'ED', 'SP', 'NA']:
                            hunk_matches_buggy = results_file[i][file_exec]['result']['hunkMatchesBuggy']
                            hunk_matches_patch = results_file[i][file_exec]['result']['hunkMatchesPatch']

                            hunk_classifications = []

                            if len(hunk_matches_buggy) == 0 or len(hunk_matches_patch)==0:
                                if len(hunk_matches_buggy) != 0:
                                    for patch_nr in hunk_matches_buggy:
                                        class_buggy = hunk_matches_buggy[patch_nr]['class']
                                        class_patch = 'MC'

                                        hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                        hunk_classifications.append(hunk_class)
                                elif len(hunk_matches_patch) != 0:
                                    for patch_nr in hunk_matches_buggy:
                                        class_buggy = 'MC'
                                        class_patch = hunk_matches_patch[patch_nr]['class']

                                        hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                        hunk_classifications.append(hunk_class)
                                else:
                                    class_buggy = 'MC'
                                    class_patch = 'MC'
                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                    hunk_classifications.append(hunk_class)

                            elif len(hunk_matches_buggy) == len(hunk_matches_patch):
                                for patch_nr in hunk_matches_buggy:
                                    class_buggy = hunk_matches_buggy[patch_nr]['class']
                                    class_patch = hunk_matches_patch[patch_nr]['class']

                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                    hunk_classifications.append(hunk_class)
                            classifications[pair_id][i][file_exec]['hunks'].append(hunk_classifications)
                        
                except Exception as e:
                    print('Pair: ', pair_id)
                    print('file: ', file_exec)
                    print('PR: ', i)
                    print(e)
#                     print(results_file[i][file_exec]['result'])
    except EOFError:
        continue

In [ ]:
total_file_classes = {'ED':0, 'MO': 0,'SP':0, 'NA':0, 'NE':0, 'CC':0, 'ERROR':0}
total_hunk_classes = {'ED':0, 'MO': 0,'SP':0, 'NA':0,}
lang_distr = {}

for pair in classifications:
    for pr in classifications[pair]:
        for file in classifications[pair][pr]:
            class_ = classifications[pair][pr][file]['class']
            if  classifications[pair][pr][file]['ext'] not in lang_distr:
                lang_distr[classifications[pair][pr][file]['ext']] = 1
            else:
                lang_distr[classifications[pair][pr][file]['ext']] += 1
                
            if class_ == 'OTHER EXT':
                class_= 'CC'
            if class_ == 'NOT EXISTING':
                class_ = 'NE'
            try:
                total_file_classes[class_] += 1
            except:
                print(class_)
            
            for h_list in classifications[pair][pr][file]['hunks']:
                for h in h_list:
                    total_hunk_classes[h] += 1

In [ ]:
lang_distr_sorted = {k: v for k, v in sorted(lang_distr.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
lang_distr_sorted_list = []
for i in lang_distr_sorted:
    lang_distr_sorted_list.append([i, lang_distr_sorted[i]])

In [ ]:
with open('languages.csv', 'w') as f:
    write = csv.writer(f)
      
    write.writerow(['Extension', 'Count'])
    write.writerows(lang_distr_sorted_list)

In [ ]:
total_hist = [total_file_classes['MO'], total_file_classes['ED'], total_file_classes['SP'], total_file_classes['NA'], total_file_classes['NE'], total_file_classes['CC'], total_file_classes['ERROR']]

left = [1, 2, 3, 4, 5, 6, 7]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI', 'NE', 'CC', 'ERROR'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3", "#ff7f00", "#ffff33", "#a65628"])

In [ ]:
total_hist = [total_hunk_classes['MO'], total_hunk_classes['ED'], total_hunk_classes['SP'], total_hunk_classes['NA']]

left = [1, 2, 3, 4]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3"])

In [ ]:
total_file_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0, 'NE':0, 'CC':0, 'ERROR':0}
total_hunk_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0,}

py_files = []

for pair in classifications:
    for pr in classifications[pair]:
        for file in classifications[pair][pr]:
            class_ = classifications[pair][pr][file]['class']
            ext = classifications[pair][pr][file]['ext']
            hunks = classifications[pair][pr][file]['hunks']
            
            if ext not in total_file_ext_classes:
                total_file_ext_classes[ext] = {}
            
            if class_ == 'OTHER EXT':
                class_= 'CC'
            if class_ == 'NOT EXISTING':
                class_ = 'NE'
                
            if ext == 'py':
                py_files.append(class_)
            try:
                total_file_ext_classes[ext][class_]+=1
            except:
                total_file_ext_classes[ext][class_] = 1

            if ext not in total_hunk_ext_classes:
                total_hunk_ext_classes[ext] = {}
            for h_list in hunks:
                for h in h_list:
                    if h not in total_hunk_ext_classes[ext]:
                        total_hunk_ext_classes[ext][h] = 1
                    total_hunk_ext_classes[ext][h] += 1

In [ ]:
def get_hist_data(file, total_file_ext_classes):
    fec = total_file_ext_classes[file]
    labels = ['MO', 'ED', 'SP', 'NA', 'NE', 'CC', 'ERROR']
    hist = []
    for i in labels:
        try:
            hist.append(fec[i])
        except:
            hist.append(0)

    return hist

def plot_hist(file1, file2, file3, total_file_ext_classes):
    names = ['MO', 'ED', 'SP', 'NI', 'NE', 'CC', 'ERROR']
    
    if len(file3)==0 and len(file2)==0:
        fig, axs = plt.subplots(1, 1, figsize=(20, 10), sharey=True)
        axs.grid()
        axs.set_xlabel(file1)
        axs.bar(names, get_hist_data(file1, total_file_ext_classes))
    else:    
        fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data(file1, total_file_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data(file2, total_file_ext_classes))
        axs[2].grid()
        axs[2].set_xlabel(file3)
        axs[2].bar(names, get_hist_data(file3, total_file_ext_classes))    
               
plot_hist('json', 'py', 'c', total_file_ext_classes)
plot_hist('rb', 'scala', 'php', total_file_ext_classes)
plot_hist('gradle', 'gemfile', 'ipynb', total_file_ext_classes)
plot_hist('xml', 'h', 'java', total_file_ext_classes)
plot_hist('sh', 'yaml', 'pl', total_file_ext_classes)
plot_hist('kt', 'yml', 'cpp', total_file_ext_classes)
plot_hist('js', '', '', total_file_ext_classes)


In [ ]:
# MO ED SP NA NE
list_mo=[]
list_ed=[]
list_sp=[]
list_na=[]
list_ne=[]
list_lang=[]

for lang in total_file_ext_classes:
    if lang in ['c', 'h', 'java', 'cpp', 'py', 'pl', 'json', 'rb', 'scala', 'sh', \
                'php', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        
        list_lang.append(lang)
        if 'MO' in total_file_ext_classes[lang]:
            list_mo.append(total_file_ext_classes[lang]['MO'])
        else:
            list_mo.append(0)
            
        if 'ED' in total_file_ext_classes[lang]:    
            list_ed.append(total_file_ext_classes[lang]['ED'])
        else:
            list_ed.append(0)
            
        if 'SP' in total_file_ext_classes[lang]:
            list_sp.append(total_file_ext_classes[lang]['SP'])
        else:
            list_sp.append(0)
            
        if 'NA' in total_file_ext_classes[lang]:
            list_na.append(total_file_ext_classes[lang]['NA'])
        else:
            list_na.append(0)
            
        if 'NE' in total_file_ext_classes[lang]:
            list_ne.append(total_file_ext_classes[lang]['NE'])
        else:
            list_ne.append(0)
            
lang_pd = pd.DataFrame()

lang_pd['Language'] = list_lang

lang_pd['MO'] = list_mo
lang_pd['ED'] = list_ed
lang_pd['SP'] = list_sp
lang_pd['NA'] = list_na
lang_pd['NE'] = list_ne

lang_pd_mo = lang_pd[['Language', 'MO']]
lang_pd_mo.sort_values(by='MO', inplace=True)

lang_pd_ed = lang_pd[['Language', 'ED']]
lang_pd_ed.sort_values(by='ED', inplace=True)

lang_pd_sp = lang_pd[['Language', 'SP']]
lang_pd_sp.sort_values(by='SP', inplace=True)

import matplotlib as mpl
mpl.style.use('default')


fig, axs = plt.subplots(1, 3, figsize=(20, 15), sharey=True)
fig.suptitle('Interesting class distribution', fontsize='20')
fig.supxlabel('Languages', fontsize='20')



axs[0].yaxis.grid()
axs[0].set_title('MO File Distribution', fontsize='20')
axs[0].set_xticklabels(labels = lang_pd_mo[lang_pd_mo['MO']!=0]['Language'], rotation=45, ha='right', fontsize='20')
axs[0].set_ylabel('File count', fontsize='20')
axs[0].set_yscale('log')


axs[1].yaxis.grid()
axs[1].set_title('ED File Distribution', fontsize='20')
axs[1].set_xticklabels(labels = lang_pd_ed[lang_pd_ed['ED']!=0]['Language'], rotation=45, ha='right', fontsize='20')
axs[1].set_yscale('log')

axs[2].yaxis.grid()
axs[2].set_title('SP File Distribution', fontsize='20')
axs[2].set_xticklabels(labels = lang_pd_sp[lang_pd_sp['SP']!=0]['Language'], rotation=45, ha='right', fontsize='20')   
axs[2].set_yscale('log')

colors_MO=[]
for i in lang_pd_mo[lang_pd_mo['MO']!=0]['Language']:
    if i in ['cpp','json', 'scala', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        colors_MO.append('tab:orange')
    else:
        colors_MO.append('b')
        
colors_ED=[]
for i in lang_pd_ed[lang_pd_ed['ED']!=0]['Language']:
    if i in ['cpp','json', 'scala', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        colors_ED.append('tab:orange')
    else:
        colors_ED.append('b')
        
colors_SP=[]
for i in lang_pd_sp[lang_pd_sp['SP']!=0]['Language']:
    if i in ['cpp','json', 'scala', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        colors_SP.append('tab:orange')
    else:
        colors_SP.append('b')
        
        
axs[0].bar(lang_pd_mo[lang_pd_mo['MO']!=0]['Language'], lang_pd_mo[lang_pd_mo['MO']!=0]['MO'], color=colors_MO)

axs[1].bar(lang_pd_ed[lang_pd_ed['ED']!=0]['Language'], lang_pd_ed[lang_pd_ed['ED']!=0]['ED'], color=colors_ED)

axs[2].bar(lang_pd_sp[lang_pd_sp['SP']!=0]['Language'], lang_pd_sp[lang_pd_sp['SP']!=0]['SP'], color=colors_SP)

fig.savefig("bugFixFileDistr.pdf")

In [ ]:
def get_hist_data_hunk(ext, total_hunk_ext_classes):
    hec = total_hunk_ext_classes[ext]
    labels = ['MO', 'ED', 'SP', 'NA']
    hist = []
    for i in labels:
        try:
            hist.append(hec[i])
        except:
            hist.append(0)

    return hist

def plot_hist(file1, file2, file3, total_hunk_ext_classes):
    names = ['MO', 'ED', 'SP', 'NI']
    
    if len(file3)==0 and len(file2)==0:
        fig, axs = plt.subplots(1, 1, figsize=(20, 10), sharey=True)
        axs.grid()
        axs.set_xlabel(file1)
        axs.bar(names, get_hist_data_hunk(file1, total_hunk_ext_classes))
    else:    
        fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data_hunk(file1, total_hunk_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data_hunk(file2, total_hunk_ext_classes))
        axs[2].grid()
        axs[2].set_xlabel(file3)
        axs[2].bar(names, get_hist_data_hunk(file3, total_hunk_ext_classes))    
               
plot_hist('c', 'py', 'java', total_hunk_ext_classes)
plot_hist('xml', 'scala', 'json', total_hunk_ext_classes)
plot_hist('php', 'cpp', 'h', total_hunk_ext_classes)
plot_hist('pl', 'kt', 'ipynb', total_hunk_ext_classes)
plot_hist('sh', 'yaml', 'rb', total_hunk_ext_classes)
plot_hist('gradle', 'yml', 'gemfile', total_hunk_ext_classes)
plot_hist('js', '', '', total_hunk_ext_classes)


## Distribution of Apache Kafka to show gain

In [ ]:
directory = 'Repos_results_bugfix'

totals_ = {}
repo_nr = 0

pair_set = set()

i = 0
prev_pair_id=4

all_pr_class_list = []
all_counts_list = []

for file in os.listdir(directory):
    pair_id = file.split('_')[0]
    pair_set.add(pair_id)
    with open(f'{directory}/{file}', 'rb') as f:
        try:
            all_results = pickle.load(f)[0]

            pr_class = totals.final_class(all_results)
            all_pr_class_list.append(pr_class)
            all_counts = totals.count_all_classifications(pr_class)
            all_counts_list.append(all_counts)
        except EOFError:
            continue
            
dict_final ={}
for i in all_pr_class_list:
    dict_final=merge_dict(dict_final, i)

all_counts_all = dict(functools.reduce(operator.add,map(collections.Counter, all_counts_list)))
totals_[pair_id] ={}
totals_[pair_id]['pr_class'] = dict_final
totals_[pair_id]['all_counts'] = all_counts_all

#             with open('Repos_totals/'+ str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_totals.pkl', 'wb') as f:
#                 pickle.dump([pr_class, all_counts],f)
#             except Exception as e:
#                 print(e)

In [ ]:
"""
    Plotting the distribution of classifications
"""

total_all = 0
total_mo_all = 0
total_ed_all = 0
total_sp_all = 0
total_cc_all = 0
total_ne_all = 0
total_error_all = 0
total_na_all = 0

all_results = []

repos = 342
for repo_file in totals_:
    try:
        variant1 = dataset_dict[int(repo_file)]['source']
        variant2 = dataset_dict[int(repo_file)]['destination']
        variant1_prs = dataset_dict[int(repo_file)]['pr'].split('/')

        all_totals= totals_[repo_file]['pr_class']

        total_NA = 0
        total_ED = 0
        total_MO = 0
        total_CC = 0
        total_SP= 0
        total_NE = 0
        total_ERROR = 0

        for pr in all_totals:
            verdict = all_totals[pr]['class']
            if verdict == 'ED':
                total_ED += 1
            elif verdict =='MO':
                total_MO += 1
            elif verdict == 'SP':
                total_SP += 1
            elif verdict == 'NA':
                total_NA += 1
            elif verdict == 'CC':
                total_CC += 1
            elif verdict =='NE':
                total_NE += 1
            elif verdict == 'ERROR':
                total_ERROR += 1
                
            total_mid = total_MO+ total_ED + total_SP
            total_all += total_mid
            
        total_total =len(variant1_prs)
        
        total_mo_all += total_MO
        total_ed_all += total_ED
        total_sp_all += total_SP
        total_na_all += total_NA

        total_cc_all += total_CC
        total_ne_all += total_NE
        total_error_all += total_ERROR

        totals_list = [total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR]

#         analysis.all_class_bar(totals_list, repo_file, variant1, variant2, False)
#         analysis.all_class_pie(totals_list, repo_file, variant1, variant2, False)

        all_results.append([variant1, variant2, repo_file, total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR])

    except Exception as e:
        print(variant1)
        print("Exception: ", e, '\n\n')

# header = ["variant1", "variant2", "repo_file", "total_MO", "total_ED", "total_SP", "total_CC", "total_NE", "total_NA"," total_ERROR"]
# with open("results_16_05_22.csv", "w") as f:
#     writer = csv.writer(f)
#     # write the header
#     writer.writerow(header)

#     # write multiple rows
#     writer.writerows(all_results)

In [ ]:
print('MO = ',total_mo_all)
print('ED = ',total_ed_all)
print('SP = ',total_sp_all)
print('NI = ', total_na_all)

print('CC = ',total_cc_all)
print('NE = ',total_ne_all)
print('ERROR = ', total_error_all)

In [ ]:
import matplotlib.pyplot as plt

total_hist = [total_mo_all, total_ed_all, total_sp_all, total_na_all, total_cc_all, total_ne_all, total_error_all]

left = [1, 2, 3, 4, 5, 6, 7]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI', 'CC', 'NE', 'ERROR'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3", "#ff7f00", "#ffff33", "#a65628"])

In [ ]:
total_classified = total_mo_all+total_ed_all+total_sp_all+total_na_all+total_cc_all+total_error_all+total_ne_all
print(f'total classified = {total_classified}')
print(f'total interesting = {100*(total_mo_all+total_ed_all+total_sp_all)/total_classified}%')
print(f'total unintersting = {100*total_na_all/total_classified}%')
print(f'total error = {100*total_error_all/total_classified}%')
print(f'total non existant = {100*total_ne_all/total_classified}%')
print(f'total cannot classify = {100*total_cc_all/total_classified}%')

In [ ]:
import Methods.classifier as classifier

classifications = {}

directory = 'Repos_results_bugfix'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in classifications:
                classifications[pair_id] = {}

            for i in results_file:
                classifications[pair_id][i] = {}

                try:
                    for file_exec in results_file[i]:
                        ext = file_exec.split('.')[-1]
                        classifications[pair_id][i][file_exec]={}
                        classifications[pair_id][i][file_exec]['ext'] = ext
                        classifications[pair_id][i][file_exec]['hunks'] = []
                        classifications[pair_id][i][file_exec]['class'] = ''
                        
                        patchClass = results_file[i][file_exec]['result']['patchClass']
                        
                        classifications[pair_id][i][file_exec]['class'] = patchClass
                        
                        if patchClass in ['MO', 'ED', 'SP', 'NA']:
                            hunk_matches_buggy = results_file[i][file_exec]['result']['hunkMatchesBuggy']
                            hunk_matches_patch = results_file[i][file_exec]['result']['hunkMatchesPatch']

                            hunk_classifications = []

                            if len(hunk_matches_buggy) == 0 or len(hunk_matches_patch)==0:
                                if len(hunk_matches_buggy) != 0:
                                    for patch_nr in hunk_matches_buggy:
                                        class_buggy = hunk_matches_buggy[patch_nr]['class']
                                        class_patch = 'MC'

                                        hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                        hunk_classifications.append(hunk_class)
                                elif len(hunk_matches_patch) != 0:
                                    for patch_nr in hunk_matches_buggy:
                                        class_buggy = 'MC'
                                        class_patch = hunk_matches_patch[patch_nr]['class']

                                        hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                        hunk_classifications.append(hunk_class)
                                else:
                                    class_buggy = 'MC'
                                    class_patch = 'MC'
                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                    hunk_classifications.append(hunk_class)

                            elif len(hunk_matches_buggy) == len(hunk_matches_patch):
                                for patch_nr in hunk_matches_buggy:
                                    class_buggy = hunk_matches_buggy[patch_nr]['class']
                                    class_patch = hunk_matches_patch[patch_nr]['class']

                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                    hunk_classifications.append(hunk_class)
                            classifications[pair_id][i][file_exec]['hunks'].append(hunk_classifications)
                        
                except Exception as e:
                    print('Pair: ', pair_id)
                    print('file: ', file_exec)
                    print('PR: ', i)
                    print(e)
#                     print(results_file[i][file_exec]['result'])
    except EOFError:
        continue

In [ ]:
total_file_classes = {'ED':0, 'MO': 0,'SP':0, 'NA':0, 'NE':0, 'CC':0, 'ERROR':0}
total_hunk_classes = {'ED':0, 'MO': 0,'SP':0, 'NA':0,}

empty_class = 0

for pair in classifications:
    for pr in classifications[pair]:
        for file in classifications[pair][pr]:
            class_ = classifications[pair][pr][file]['class']
            if class_ == 'OTHER EXT':
                class_= 'CC'
            if class_ == 'NOT EXISTING':
                class_ = 'NE'
            try:
                total_file_classes[class_] += 1
            except:
                empty_class += 1
            for h_list in classifications[pair][pr][file]['hunks']:
                for h in h_list:
                    total_hunk_classes[h] += 1

In [ ]:
total_hist = [total_file_classes['MO'], total_file_classes['ED'], total_file_classes['SP'], total_file_classes['NA'], total_file_classes['NE'], total_file_classes['CC'], total_file_classes['ERROR']]

left = [1, 2, 3, 4, 5, 6, 7]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI', 'NE', 'CC', 'ERROR'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3", "#ff7f00", "#ffff33", "#a65628"])

In [ ]:
total_hist = [total_hunk_classes['MO'], total_hunk_classes['ED'], total_hunk_classes['SP'], total_hunk_classes['NA']]

left = [1, 2, 3, 4]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3"])

In [ ]:
total_file_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0, 'NE':0, 'CC':0, 'ERROR':0}
total_hunk_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0,}

py_files = []

for pair in classifications:
    for pr in classifications[pair]:
        for file in classifications[pair][pr]:
            class_ = classifications[pair][pr][file]['class']
            ext = classifications[pair][pr][file]['ext']
            hunks = classifications[pair][pr][file]['hunks']
            
            if ext not in total_file_ext_classes:
                total_file_ext_classes[ext] = {}
            
            if class_ == 'OTHER EXT':
                class_= 'CC'
            if class_ == 'NOT EXISTING':
                class_ = 'NE'
                
            if ext == 'py':
                py_files.append(class_)
            try:
                total_file_ext_classes[ext][class_]+=1
            except:
                total_file_ext_classes[ext][class_] = 1

            if ext not in total_hunk_ext_classes:
                total_hunk_ext_classes[ext] = {}
            for h_list in hunks:
                for h in h_list:
                    if h not in total_hunk_ext_classes[ext]:
                        total_hunk_ext_classes[ext][h] = 1
                    total_hunk_ext_classes[ext][h] += 1

In [ ]:
def get_hist_data(file, total_file_ext_classes):
    fec = total_file_ext_classes[file]
    labels = ['MO', 'ED', 'SP', 'NA', 'NE', 'CC', 'ERROR']
    hist = []
    for i in labels:
        try:
            hist.append(fec[i])
        except:
            hist.append(0)

    return hist

def plot_hist(file1, file2, file3, total_file_ext_classes):
    names = ['MO', 'ED', 'SP', 'NI', 'NE', 'CC', 'ERROR']
    
    if len(file3)==0 and len(file2)==0:
        fig, axs = plt.subplots(1, 1, figsize=(20, 10), sharey=True)
        axs.grid()
        axs.set_xlabel(file1)
        axs.bar(names, get_hist_data(file1, total_file_ext_classes))
    else:    
        fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data(file1, total_file_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data(file2, total_file_ext_classes))
        axs[2].grid()
        axs[2].set_xlabel(file3)
        axs[2].bar(names, get_hist_data(file3, total_file_ext_classes))    
               
plot_hist('py', 'java', 'sh', total_hunk_ext_classes)
plot_hist('xml', 'scala', 'json', total_hunk_ext_classes)
plot_hist('yaml', 'gradle', 'yml', total_hunk_ext_classes)


In [ ]:
def get_hist_data_hunk(ext, total_hunk_ext_classes):
    hec = total_hunk_ext_classes[ext]
    labels = ['MO', 'ED', 'SP', 'NA']
    hist = []
    for i in labels:
        try:
            hist.append(hec[i])
        except:
            hist.append(0)

    return hist

def plot_hist(file1, file2, file3, total_hunk_ext_classes):
    names = ['MO', 'ED', 'SP', 'NI']
    
    if len(file3)==0 and len(file2)==0:
        fig, axs = plt.subplots(1, 1, figsize=(20, 10), sharey=True)
        axs.grid()
        axs.set_xlabel(file1)
        axs.bar(names, get_hist_data_hunk(file1, total_hunk_ext_classes))
    else:    
        fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data_hunk(file1, total_hunk_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data_hunk(file2, total_hunk_ext_classes))
        axs[2].grid()
        axs[2].set_xlabel(file3)
        axs[2].bar(names, get_hist_data_hunk(file3, total_hunk_ext_classes))    
               
plot_hist('py', 'java', 'sh', total_hunk_ext_classes)
plot_hist('xml', 'scala', 'json', total_hunk_ext_classes)
plot_hist('yaml', 'gradle', 'yml', total_hunk_ext_classes)



In [ ]:
total_hunk_ext_classes

In [ ]:
len(list(classifications['4'].keys()))

In [ ]:
# keys from previous run
run1 = ['8102', '8106', '8109', '8111', '8113', '8116', '8119', '8124', '8126', '8130', '8133', '8138', '8140', '8142', '8145', '8154', '8156', '8158', '8165', '8170', '8177', '8178', '8179', '8180', '8181', '8186', '8195', '8197', '8198', '8200', '8203', '8209', '8224', '8226', '8227', '8232', '8233', '8234', '8239', '8241', '8243', '8247', '8256', '8257', '8258', '8259', '8260', '8272', '8273', '8280', '8285', '8287', '8289', '8290', '8291', '8298', '8300', '8301', '8302', '8305', '8310', '8314', '8317', '8319', '8324', '8327', '8330', '8341', '8346', '8348', '8360', '8367', '8382', '8389', '8390', '8392', '8393', '8399', '8400', '8403', '8411', '8412', '8416', '8417', '8419', '8420', '8423', '8424', '8428', '8432', '8435', '8436', '8442', '8447', '8452', '8454', '8459', '8463', '8465', '8476', '8477', '8478', '8479', '8488', '8492', '8493', '8502', '8503', '8504', '8507', '8515', '8524', '8532', '8533', '8536', '8548', '8549', '8552', '8557', '8560', '8561', '8562', '8566', '8567', '8571', '8572', '8579', '8580', '8581', '8584', '8586', '8587', '8599', '8600', '8603', '8609', '8618', '8623', '8625', '8628', '8635', '8646', '8651', '8658', '8659', '8662', '8665', '8672', '8675', '8689', '8693', '8700', '8702', '8705', '8708', '8712', '8716', '8720', '8728', '8730', '8731', '8735', '8743', '8747', '8748', '8749', '8750', '8764', '8772', '8774', '8777', '8779', '8782', '8785', '8786', '8789', '8790', '8797', '8799', '8800', '8802', '8804', '8805', '8809', '8811', '8812', '8815', '8817', '8822', '8823', '8827', '8829', '8841', '8843', '8844', '8848', '8854', '8857', '8858', '8865', '8866', '8870', '8872', '8873', '8874', '8880', '8883', '8884', '8886', '8894', '8900', '8905', '8910', '8913', '8914', '8917', '8918', '8919', '8922', '8927', '8929', '8944', '8951', '8956', '8959', '8963', '8973', '8980', '8981', '8983', '8985', '8987', '8990', '8998', '9003', '9020', '9022', '9023', '9027', '9029', '9033', '9034', '9053', '9056', '9059', '9061', '9071', '9075', '9079', '9086', '9087', '9091', '9095', '9106', '9112', '9121', '9122', '9127', '9140', '9142', '9143', '9153', '9163', '9166', '9172', '9178', '9180', '9183', '9184', '9191', '9198', '9199', '9202', '9203', '9205', '9217', '9218', '9219', '9236', '9245', '9262', '9265', '9267', '9277', '9279', '9280', '9284', '9285', '9294', '9295', '9299', '9301', '9304', '9305', '9308', '9309', '9312', '9313', '9321', '9323', '9330', '9334', '9341', '9344', '9354', '9356', '9357', '9364', '9372', '9374', '9375', '9385', '9390', '9391', '9392', '9393', '9407', '9413', '9417', '9423', '9425', '9426', '9428', '9431', '9432', '9433', '9443', '9446', '9447', '9450', '9458', '9460', '9462', '9466', '9479', '9486', '9487', '9489', '9503', '9508', '9521', '9524', '9530', '9538', '9541', '9544', '9559', '9562', '9563', '9568', '9570', '9586', '9596', '9598', '9599', '9603', '9605', '9614', '9629', '9636', '9649', '9655', '9665', '9671', '9673', '9676', '9678', '9681', '9682', '9683', '9688', '9691', '9718', '9723', '9730', '9733', '9745', '9751', '9752', '9753', '9757', '9759', '9762', '9765', '9768', '9776', '9778', '9781', '9784', '9792', '9800', '9807', '9810', '9812', '9818', '9821', '9827', '9834', '9837', '9843', '9862', '9873', '9875', '9885', '9887', '9893', '9897', '9906', '9911', '9923', '9930', '9935', '9939', '9940', '9942', '9943', '9947', '9948', '9950', '9956', '9965', '9966', '9968', '9974', '9978', '9987', '9990', '10000', '10006', '10014', '10017', '10020', '10024', '10029', '10032', '10033', '10034', '10042', '10052', '10059', '10060', '10068', '10072', '10073', '10074', '10082', '10087', '10088', '10102', '10103', '10104', '10114', '10117', '10127', '10130', '10133', '10134', '10136', '10141', '10143', '10144', '10146', '10153', '10158', '10181', '10193', '10196', '10204', '10210', '10213', '10223', '10226', '10231', '10237', '10243', '10250', '10260', '10264', '10268', '10272', '10292', '10297', '10301', '10304', '10312', '10320', '10321', '10322', '10331', '10334', '10344', '10355', '10361', '10364', '10369', '10372', '10374', '10386', '10387', '10392', '10396', '10398', '10400', '10402', '10404', '10407', '10413', '10423', '10433', '10444', '10445', '10449', '10452', '10453', '10462', '10469', '10474', '10486', '10495', '10503', '10505', '10517', '10529', '10536', '10551', '10553', '10558', '10560', '10561', '10571', '10575', '10584', '10586', '10597', '10604', '10611', '10612', '10621', '10630', '10631', '10632', '10639', '10643', '10657', '10688', '10693', '10694', '10703', '10704', '10706', '10707', '10709', '10713', '10717', '10719', '10735', '10742', '10749', '10759', '10766', '10770', '10776', '10777', '10780', '10794', '10795', '10796', '10797', '10809', '10818', '10822', '10836', '10843', '10858', '10860', '10861', '10877', '10879', '10883', '10885', '10886', '10888', '10890', '10901', '10908', '10915', '10916', '10917', '10923', '10925', '10939', '10941', '10943', '10945', '10958', '10966', '10974', '10975', '10984', '10985', '10989', '10991', '10995', '10998', '11001', '11016', '11019', '11021', '11022', '11026', '11035', '11043', '11048', '11055', '11056', '11065', '11069', '11071', '11090', '11091', '11092', '11093', '11095', '11103', '11108', '11111', '11118', '11122', '11129']

run2 = ['8700', '8702', '8705', '8708', '8712', '8716', '8720', '8728', '8730', '8731', '8735', '8743', '8747', '8748', '8749', '8750', '8764', '8772', '8774', '8777', '8779', '8782', '8785', '8786', '8789', '8790', '8797', '8799', '8800', '8802', '8804', '8805', '8809', '8811', '8812', '8815', '8817', '8822', '8823', '8827', '8829', '8841', '8843', '8844', '8848', '8854', '8857', '8858', '8865', '8866', '8870', '8872', '8873', '8874', '8880', '8883', '8884', '8886', '8894', '8900', '8905', '8910', '8913', '8914', '8917', '8918', '8919', '8922', '8927', '8929', '8944', '8951', '8956', '8959', '8963', '8973', '8980', '8981', '8983', '8985', '8987', '8990', '8998', '9003', '9020', '9305', '9308', '9309', '9312', '9313', '9321', '9323', '9330', '9334', '9341', '9344', '9354', '9356', '9357', '9364', '9372', '9374', '9375', '9385', '9390', '9391', '9392', '9393', '9407', '9413', '9417', '9423', '9425', '9426', '9428', '9431', '9432', '9433', '9443', '9446', '9447', '9450', '9458', '9460', '9462', '9466', '9479', '9486', '9487', '9489', '9503', '9508', '9521', '9524', '9530', '9538', '9541', '9544', '9559', '9562', '9563', '9568', '9570', '9586', '9596', '9598', '9599', '9603', '9605', '9614', '9629', '9636', '9649', '9655', '9665', '9671', '9673', '9676', '9678', '9681', '9682', '9683', '9688', '9691', '9718', '9723', '9730', '9733', '9745', '9751', '9752', '9753', '9757', '9759', '9762', '9765', '9768', '9776', '9778', '9781', '9784', '9792', '9800', '9807', '9810', '9812', '9818', '9821', '9827', '9834', '9837']

commonPrs = set(run1).intersection(run2)

In [ ]:
comparison_details = []

for i in run2:
    comparison_details.append([i,totals_['4']['pr_class'][i]['class']])

In [ ]:
header = ["PR run2", "class run2"]
with open("comparisonApacheKafka2.csv", "w") as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(comparison_details)

# Find distribution of languages in different pairs and select 1 to illustrate the language gain

In [ ]:
pair_lang = {}

directory = 'Repos_results_bugfix'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in pair_lang:
                pair_lang[pair_id] = {}
                pair_lang[pair_id]['extensions'] = {}
                pair_lang[pair_id]['all counts'] = []
                
            for i in results_file:
                try:
                    for file_exec in results_file[i]:
                        ext = file_exec.split('.')[-1]
                        if ext not in pair_lang[pair_id]:
                            pair_lang[pair_id]['extensions'][ext] = 1
                        else:
                            pair_lang[pair_id]['extensions'][ext] += 1
                except Exception as e:
                    print('Pair: ', pair_id)
                    print('file: ', file_exec)
                    print('PR: ', i)
                    print(e)
        
            pr_class = totals.final_class(results_file)
            all_counts = totals.count_all_classifications(pr_class)
            pair_lang[pair_id]['all counts'] = all_counts  
    except EOFError:
        continue

In [ ]:
pair_lang

In [ ]:
pair_lang_distr = {}

directory = 'Repos_results_bugfix'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in pair_lang_distr:
                pair_lang_distr[pair_id] = {}
                pair_lang_distr[pair_id]['extensions'] = {
                    'json':0,
                    'php': 0,
                    'py':0,
                    'c':0,
                    'rb':0,
                    'scala':0,
                    'gradle':0,
                    'gemfile':0,
                    'Gemfile':0,
                    'xml':0,
                    'h':0,
                    'java':0,
                    'sh':0,
                    'yaml':0,
                    'pl':0,
                    'kt':0,
                    'yml':0,
                    'cpp':0,
                    'js':0,
                }
                pair_lang_distr[pair_id]['all counts'] = []
            
            for i in results_file:
                try:
                    for file_exec in results_file[i]:
                        ext = file_exec.split('.')[-1]
                        
                    if ext in pair_lang_distr[pair_id]['extensions']:
                        pair_lang_distr[pair_id]['extensions'][ext] += 1
                        
                except Exception as e:
                    print('Pair: ', pair_id)
                    print('file: ', file_exec)
                    print('PR: ', i)
                    print(e)
        
            pr_class = totals.final_class(results_file)
            all_counts = totals.count_all_classifications(pr_class)
            pair_lang_distr[pair_id]['all counts'] = all_counts  
    except EOFError:
        continue

In [ ]:
pair_lang_distr_filter = []
for p in pair_lang_distr:
    sum_interesting = pair_lang_distr[p]['all counts']['MO'] +  pair_lang_distr[p]['all counts']['ED'] + pair_lang_distr[p]['all counts']['SP']
    
    if sum_interesting > 6:
        pair_lang_distr_filter.append([p,pair_lang_distr[p]])

In [ ]:
pair_lang_distr_filter

In [ ]:
with open("Repos_prs_bugfix/11"+ '_' + dataset_dict[11]['source'].split('/')[0] + '_' + dataset_dict[11]['source'].split('/')[1] + '_prs.pkl', 'rb') as f:
    prs = pickle.load(f)


In [ ]:
dataset_dict[11]

# Language gain example

In [ ]:
def merge_dict(dict1, dict2):
    return {**dict1, **dict2}

In [ ]:
directory = 'Language_gain/Repos_results_concrete_example'

totals_ = {}
repo_nr = 0

pair_set = set()

i = 0
prev_pair_id=11

all_pr_class_list = []
all_counts_list = []

for file in os.listdir(directory):
    pair_id = file.split('_')[0]
    pair_set.add(pair_id)
    with open(f'{directory}/{file}', 'rb') as f:
        try:
            all_results = pickle.load(f)[0]

            pr_class = totals.final_class(all_results)
            all_pr_class_list.append(pr_class)
            all_counts = totals.count_all_classifications(pr_class)
            all_counts_list.append(all_counts)
        except EOFError:
            continue
            
dict_final ={}
for i in all_pr_class_list:
    dict_final=merge_dict(dict_final, i)

all_counts_all = dict(functools.reduce(operator.add,map(collections.Counter, all_counts_list)))
totals_[pair_id] ={}
totals_[pair_id]['pr_class'] = dict_final
totals_[pair_id]['all_counts'] = all_counts_all

In [ ]:
totals_['11']['all_counts']

In [ ]:
new_results = 'Repos_results_bugfix/11_linkedin_kafka_results.pkl'

with open(new_results, 'rb') as f:
    all_results_new = pickle.load(f)[0]
    pr_class_new = totals.final_class(all_results_new)
    all_counts_new = totals.count_all_classifications(pr_class_new)

In [ ]:
totals_new = {}
totals_new['11'] ={}
totals_new['11']['pr_class'] = pr_class_new
totals_new['11']['all_counts'] = all_counts_new

In [ ]:
totals_new['11']['all_counts']

In [ ]:
header = ['PR', 'PrevClass', 'NewClass', 'Gain']

keys = totals_['11']['pr_class'].keys()
csv_data = []
for i in keys:
    pr = i
    
    pr_old_class = totals_['11']['pr_class'][i]['class']
    pr_new_class = totals_new['11']['pr_class'][i]['class']
    
    gain = False
    if pr_old_class == 'CC' and pr_new_class in ['ED', 'MO', 'SP']:
        gain = True
    
    csv_data.append([pr, pr_old_class, pr_new_class, gain])
    
# with open('language_gain_apache_kafka.csv', 'w') as f:
#     write = csv.writer(f)
      
#     write.writerow(header)
#     write.writerows(csv_data)

In [ ]:
import pandas as pd
language_gain_df=pd.DataFrame(csv_data, columns=header)

In [ ]:
language_gain_df[language_gain_df['PrevClass']=='NE'][language_gain_df['NewClass']!='NE']

In [ ]:
all_results_new['114']

In [ ]:
totals_['11']['pr_class']['243']

In [ ]:
totals_new['11']['pr_class']['243']

In [ ]:
old_counts = list(totals_['11']['all_counts'].values())
new_counts = list(totals_new['11']['all_counts'].values())

In [ ]:
print(list(totals_['11']['all_counts'].keys()))
print(old_counts)
print(list(totals_new['11']['all_counts'].keys()))
print(new_counts)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

y0 = [44, 5, 3, 18, 52, 39]
y1 = [70, 7, 2, 43,  1, 37]

x = 2
width = 0.3

w0 = np.arange(len(y0))
w1 = [x + width for x in w0]

plt.figure(figsize=(15,10), dpi=80)

patch1 = mpatches.Patch(color="b", label='Run-1')
patch2 = mpatches.Patch(color="turquoise", label='Run-2')
plt.legend(fontsize=18, loc="upper left", handles = [patch1, patch2])

plt.bar(w0, y0, color='b', width=width, edgecolor='white', label='Run-1')
plt.bar(w1, y1, color='turquoise', width=width, edgecolor='white', label='Run-2')
plt.grid(True)
plt.legend(loc='upper right', title='source')

plt.xlabel('Patch Classifications', fontsize=20)
plt.title('Apache Kafka Language Gain (Run-1 vs Run-2)', fontsize=20)
plt.ylabel('Patch count', fontsize=20)
plt.xticks(fontsize = 14)   
plt.yticks(fontsize = 14) 
plt.xticks([r + width for r in range(len(y0))], 
           ['MO', 'ED', 'SP', 'NI', 'CC', 'NE'])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df2=pd.DataFrame([['MO','Run-1', 44], ['ED', 'Run-1', 5], ['SP','Run-1', 3] , ['NI', 'Run-1', 18], ['CC', 'Run-1',52], ['NE','Run-1', 39], \
                  ['MO','Run-2', 70], ['ED', 'Run-2', 7], ['SP','Run-2', 2] , ['NI', 'Run-2', 43], ['CC', 'Run-2', 1], ['NE','Run-2', 37]] ,columns=['class', 'source', 'apache'])
#set seaborn plotting aesthetics

# sns.set()
sns.set(style='white', font_scale = 1.5)
# df2 = pd.read_csv('dataset1.csv',sep=';')
#create grouped bar chart
g = sns.barplot(x='class', y='apache', palette='bright', hue='source', data=df2,
            # palette=[colors[1], colors[5]]
               )

#add overall title
#add axis titles
plt.xlabel('Classifications')
plt.ylabel('Number of patches')
plt.title('Apache Kafka Language Gain (Run-1 vs Run-2)')
plt.grid(True)
plt.legend(loc='upper right', title='source')
g.figure.set_size_inches(9,5)

ax = plt.gca()
ax.grid(which='major', axis='y', linestyle='-')
ax.grid(which='major', axis='x', linestyle='')

gg = g.get_figure()
gg.savefig("apache.pdf")

In [ ]:
import Methods.classifier as classifier

classifications = {}

directory = 'Repos_results_bugfix'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in classifications:
                classifications[pair_id] = {}

            for i in results_file:
                classifications[pair_id][i] = {}

                try:
                    for file_exec in results_file[i]:
                        ext = file_exec.split('.')[-1]
                        classifications[pair_id][i][file_exec]={}
                        classifications[pair_id][i][file_exec]['ext'] = ext
#                         classifications[pair_id][i][file_exec]['hunks'] = []
                        classifications[pair_id][i][file_exec]['class'] = ''
                        
                        patchClass = results_file[i][file_exec]['result']['patchClass']
                        
                        classifications[pair_id][i][file_exec]['class'] = patchClass
                        
#                         if patchClass in ['MO', 'ED', 'SP', 'NA']:
#                             hunk_matches_buggy = results_file[i][file_exec]['result']['hunkMatchesBuggy']
#                             hunk_matches_patch = results_file[i][file_exec]['result']['hunkMatchesPatch']

#                             hunk_classifications = []

#                             if len(hunk_matches_buggy) == 0 or len(hunk_matches_patch)==0:
#                                 if len(hunk_matches_buggy) != 0:
#                                     for patch_nr in hunk_matches_buggy:
#                                         class_buggy = hunk_matches_buggy[patch_nr]['class']
#                                         class_patch = 'MC'

#                                         hunk_class = classifier.classify_hunk(class_buggy, class_patch)
#                                         hunk_classifications.append(hunk_class)
#                                 elif len(hunk_matches_patch) != 0:
#                                     for patch_nr in hunk_matches_buggy:
#                                         class_buggy = 'MC'
#                                         class_patch = hunk_matches_patch[patch_nr]['class']

#                                         hunk_class = classifier.classify_hunk(class_buggy, class_patch)
#                                         hunk_classifications.append(hunk_class)
#                                 else:
#                                     class_buggy = 'MC'
#                                     class_patch = 'MC'
#                                     hunk_class = classifier.classify_hunk(class_buggy, class_patch)
#                                     hunk_classifications.append(hunk_class)

#                             elif len(hunk_matches_buggy) == len(hunk_matches_patch):
#                                 for patch_nr in hunk_matches_buggy:
#                                     class_buggy = hunk_matches_buggy[patch_nr]['class']
#                                     class_patch = hunk_matches_patch[patch_nr]['class']

#                                     hunk_class = classifier.classify_hunk(class_buggy, class_patch)
#                                     hunk_classifications.append(hunk_class)
#                             classifications[pair_id][i][file_exec]['hunks'].append(hunk_classifications)
                        
                except Exception as e:
                    print('Pair: ', pair_id)
                    print('file: ', file_exec)
                    print('PR: ', i)
                    print(e)
#                     print(results_file[i][file_exec]['result'])
    except EOFError:
        continue
        
total_file_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0, 'NE':0, 'CC':0, 'ERROR':0}
# total_hunk_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0,}

py_files = []

for pair in classifications:
    for pr in classifications[pair]:
        for file in classifications[pair][pr]:
            class_ = classifications[pair][pr][file]['class']
            ext = classifications[pair][pr][file]['ext']
#             hunks = classifications[pair][pr][file]['hunks']
            
            if ext not in total_file_ext_classes:
                total_file_ext_classes[ext] = {}
            
            if class_ == 'OTHER EXT':
                class_= 'CC'
            if class_ == 'NOT EXISTING':
                class_ = 'NE'
                
            try:
                total_file_ext_classes[ext][class_]+=1
            except:
                total_file_ext_classes[ext][class_] = 1

#             if ext not in total_hunk_ext_classes:
#                 total_hunk_ext_classes[ext] = {}
#             for h_list in hunks:
#                 for h in h_list:
#                     if h not in total_hunk_ext_classes[ext]:
#                         total_hunk_ext_classes[ext][h] = 1
#                     total_hunk_ext_classes[ext][h] += 1
                    

def get_hist_data(file, total_file_ext_classes):
    fec = total_file_ext_classes[file]
    labels = ['MO', 'ED', 'SP', 'NA', 'NE', 'CC', 'ERROR']
    hist = []
    for i in labels:
        try:
            hist.append(fec[i])
        except:
            hist.append(0)

    return hist

def plot_hist(file1, file2, file3, total_file_ext_classes):
    names = ['MO', 'ED', 'SP', 'NI', 'NE', 'CC', 'ERROR']
    
    if len(file3)==0 and len(file2)==0:
        fig, axs = plt.subplots(1, 1, figsize=(20, 10), sharey=True)
        axs.grid()
        axs.set_xlabel(file1)
        axs.bar(names, get_hist_data(file1, total_file_ext_classes))
    else:    
        fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data(file1, total_file_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data(file2, total_file_ext_classes))
        axs[2].grid()
        axs[2].set_xlabel(file3)
        axs[2].bar(names, get_hist_data(file3, total_file_ext_classes))    

In [ ]:
plot_hist('py', 'java', 'sh', total_hunk_ext_classes)
plot_hist('xml', 'scala', 'json', total_hunk_ext_classes)
plot_hist('yaml', 'gradle', 'yml', total_hunk_ext_classes)


# Optimization patches

In [ ]:
def merge_dict(dict1, dict2):
    return {**dict1, **dict2}


with open("sorted_dataset_opt.pkl", "rb") as f:
    dataset_dict = pickle.load(f)
    
directory = 'Repos_results'

totals_ = {}
repo_nr = 0

pair_set = set()
prs_set={}

i = 0
prev_pair_id = '0'

all_pr_class_list = []
all_counts_list = []

for file in os.listdir(directory):
    pair_id = file.split('_')[0]
    pair_set.add(pair_id)
    
    if pair_id not in prs_set:
        prs_set[pair_id] = set()
    
    if pair_id != prev_pair_id:
        dict_final ={}
        for i in all_pr_class_list:
            dict_final=merge_dict(dict_final, i)
        try:
            all_counts_all = dict(functools.reduce(operator.add,map(collections.Counter, all_counts_list)))
            totals_[pair_id] ={}
            totals_[pair_id]['pr_class'] = dict_final
            totals_[pair_id]['all_counts'] = all_counts_all
        except:
            totals_[pair_id] ={}
            totals_[pair_id]['pr_class'] = dict_final
            totals_[pair_id]['all_counts'] = []
            
        all_pr_class_list = []
        all_counts_list = []
        
        prev_pair_id = pair_id
    with open(f'{directory}/{file}', 'rb') as f:
        try:
            all_results = pickle.load(f)[0]

            pr_class = totals.final_class(all_results)
            all_pr_class_list.append(pr_class)
            all_counts = totals.count_all_classifications(pr_class)
            all_counts_list.append(all_counts)
            
            for i in all_results:
                prs_set[pair_id].add(i)
                
        except EOFError:
            continue
#             with open('Repos_totals/'+ str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_totals.pkl', 'wb') as f:
#                 pickle.dump([pr_class, all_counts],f)
#             except Exception as e:
#                 print(e)

In [ ]:
len_ = 0 
for i in prs_set:
    len_+=len(prs_set[i])
len_

In [ ]:
totals_

In [ ]:
import re
#Create mapping between original file and file created by me
directory = 'Repos_results/'

with open('New_data/dataset_opt.pkl', 'rb') as orig:
    data_all = pickle.load(orig)
        
mapping = {}
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):        
        name_splitted = filename.split('_')
        pair_id = int(re.findall(r'\d+', name_splitted[0])[0])

        source = dataset_dict[pair_id]['source']
        dest = dataset_dict[pair_id]['destination']
            
        mapping[pair_id] = {}
        for repo_pair in data_all:
            if data_all[repo_pair]['mainline'] == source:
                if data_all[repo_pair]['variant'] == dest:
                    mapping[pair_id]['repo_id'] = repo_pair
                    mapping[pair_id]['mainline-variant'] = True
            elif data_all[repo_pair]['mainline'] == dest:
                if data_all[repo_pair]['variant'] == source:
                    mapping[pair_id]['repo_id'] = repo_pair
                    
                    mapping[pair_id]['mainline-variant'] = False

In [ ]:
"""
    Plotting the distribution of classifications
"""

total_all = 0
total_mo_all = 0
total_ed_all = 0
total_sp_all = 0
total_cc_all = 0
total_ne_all = 0
total_error_all = 0
total_na_all = 0

all_results = []

repos = 342
for repo_file in totals_:
    try:
        variant1 = dataset_dict[int(repo_file)]['source']
        variant2 = dataset_dict[int(repo_file)]['destination']
        variant1_prs = dataset_dict[int(repo_file)]['pr'].split('/')
        
        mainlineToVariant = mapping[int(repo_file)]['mainline-variant']
        all_totals= totals_[repo_file]['pr_class']

        total_NA = 0
        total_ED = 0
        total_MO = 0
        total_CC = 0
        total_SP= 0
        total_NE = 0
        total_ERROR = 0

        for pr in all_totals:
            verdict = all_totals[pr]['class']
            if verdict == 'ED':
                total_ED += 1
            elif verdict =='MO':
                total_MO += 1
            elif verdict == 'SP':
                total_SP += 1
            elif verdict == 'NA':
                total_NA += 1
            elif verdict == 'CC':
                total_CC += 1
            elif verdict =='NE':
                total_NE += 1
            elif verdict == 'ERROR':
                total_ERROR += 1
                
            total_mid = total_MO+ total_ED + total_SP
            total_all += total_mid
            
        total_total =len(variant1_prs)
        
        total_mo_all += total_MO
        total_ed_all += total_ED
        total_sp_all += total_SP
        total_na_all += total_NA

        total_cc_all += total_CC
        total_ne_all += total_NE
        total_error_all += total_ERROR

        totals_list = [total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR]

#         analysis.all_class_bar(totals_list, repo_file, variant1, variant2, False)
#         analysis.all_class_pie(totals_list, repo_file, variant1, variant2, False)

        all_results.append([variant1, variant2, mainlineToVariant, repo_file, total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR])

    except Exception as e:
        print(variant1)
        print("Exception: ", e, '\n')

# header = ["variant1", "variant2","mainlineToVariant", "repo_file", "total_MO", "total_ED", "total_SP", "total_CC", "total_NE", "total_NA"," total_ERROR"]
# with open("results_opt_29_05_22.csv", "w") as f:
#     writer = csv.writer(f)
#     # write the header
#     writer.writerow(header)

#     # write multiple rows
#     writer.writerows(all_results)

In [ ]:
print('MO = ',total_mo_all)
print('ED = ',total_ed_all)
print('SP = ',total_sp_all)
print('NI = ', total_na_all)

print('CC = ',total_cc_all)
print('NE = ',total_ne_all)
print('ERROR = ', total_error_all)

In [ ]:
import matplotlib.pyplot as plt

total_hist = [total_mo_all, total_ed_all, total_sp_all, total_na_all, total_cc_all, total_ne_all, total_error_all]

left = [1, 2, 3, 4, 5, 6, 7]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI', 'CC', 'NE', 'ERROR'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3", "#ff7f00", "#ffff33", "#a65628"])

In [ ]:
total_classified = total_mo_all+total_ed_all+total_sp_all+total_na_all+total_cc_all+total_ne_all
print(f'total classified = {total_classified}')
print(f'total interesting = {100*(total_mo_all+total_ed_all+total_sp_all)/total_classified}%')
print(f'total unintersting = {100*total_na_all/total_classified}%')
print(f'total error = {100*total_error_all/total_classified}%')
print(f'total non existant = {100*total_ne_all/total_classified}%')
print(f'total cannot classify = {100*total_cc_all/total_classified}%')

# File classification distribution

In [ ]:
import Methods.classifier as classifier

classifications = {}

directory = 'Repos_results'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in classifications:
                classifications[pair_id] = {}

            for i in results_file:
                classifications[pair_id][i] = {}

                try:
                    for file_exec in results_file[i]:
                        ext = file_exec.split('.')[-1]
                        classifications[pair_id][i][file_exec]={}
                        classifications[pair_id][i][file_exec]['ext'] = ext
                        classifications[pair_id][i][file_exec]['hunks'] = []
                        classifications[pair_id][i][file_exec]['class'] = ''
                        
                        patchClass = results_file[i][file_exec]['result']['patchClass']
                        
                        classifications[pair_id][i][file_exec]['class'] = patchClass
                        
                        if patchClass in ['MO', 'ED', 'SP', 'NA']:
                            hunk_matches_buggy = results_file[i][file_exec]['result']['hunkMatchesBuggy']
                            hunk_matches_patch = results_file[i][file_exec]['result']['hunkMatchesPatch']

                            hunk_classifications = []

                            if len(hunk_matches_buggy) == 0 or len(hunk_matches_patch)==0:
                                if len(hunk_matches_buggy) != 0:
                                    for patch_nr in hunk_matches_buggy:
                                        class_buggy = hunk_matches_buggy[patch_nr]['class']
                                        class_patch = 'MC'

                                        hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                        hunk_classifications.append(hunk_class)
                                elif len(hunk_matches_patch) != 0:
                                    for patch_nr in hunk_matches_buggy:
                                        class_buggy = 'MC'
                                        class_patch = hunk_matches_patch[patch_nr]['class']

                                        hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                        hunk_classifications.append(hunk_class)
                                else:
                                    class_buggy = 'MC'
                                    class_patch = 'MC'
                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                    hunk_classifications.append(hunk_class)

                            elif len(hunk_matches_buggy) == len(hunk_matches_patch):
                                for patch_nr in hunk_matches_buggy:
                                    class_buggy = hunk_matches_buggy[patch_nr]['class']
                                    class_patch = hunk_matches_patch[patch_nr]['class']

                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                    hunk_classifications.append(hunk_class)
                            classifications[pair_id][i][file_exec]['hunks'].append(hunk_classifications)
                        
                except Exception as e:
                    print('Pair: ', pair_id)
                    print('file: ', file_exec)
                    print('PR: ', i)
                    print(e)
#                     print(results_file[i][file_exec]['result'])
    except EOFError:
        continue

In [ ]:
total_file_classes = {'ED':0, 'MO': 0,'SP':0, 'NA':0, 'NE':0, 'CC':0, 'ERROR':0}
total_hunk_classes = {'ED':0, 'MO': 0,'SP':0, 'NA':0,}
lang_distr = {}

for pair in classifications:
    for pr in classifications[pair]:
        for file in classifications[pair][pr]:
            class_ = classifications[pair][pr][file]['class']
            if  classifications[pair][pr][file]['ext'] not in lang_distr:
                lang_distr[classifications[pair][pr][file]['ext']] = 1
            else:
                lang_distr[classifications[pair][pr][file]['ext']] += 1
                
            if class_ == 'OTHER EXT':
                class_= 'CC'
            if class_ == 'NOT EXISTING':
                class_ = 'NE'
            try:
                total_file_classes[class_] += 1
            except:
                print(class_)
            
            for h_list in classifications[pair][pr][file]['hunks']:
                for h in h_list:
                    total_hunk_classes[h] += 1

In [ ]:
total_hist = [total_file_classes['MO'], total_file_classes['ED'], total_file_classes['SP'], total_file_classes['NA'], total_file_classes['NE'], total_file_classes['CC'], total_file_classes['ERROR']]

left = [1, 2, 3, 4, 5, 6, 7]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI', 'NE', 'CC', 'ERROR'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3", "#ff7f00", "#ffff33", "#a65628"])

In [ ]:
total_hist = [total_hunk_classes['MO'], total_hunk_classes['ED'], total_hunk_classes['SP'], total_hunk_classes['NA']]

left = [1, 2, 3, 4]

plt.figure(figsize=(15,15), dpi=80)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.bar(left, total_hist, tick_label = ['MO', 'ED', 'SP', 'NI'], width = 0.8, color = ["#e41a1c","#377eb8","#4daf4a","#984ea3"])

In [ ]:
total_file_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0, 'NE':0, 'CC':0, 'ERROR':0}
total_hunk_ext_classes = {} #{'ED':0, 'MO': 0,'SP':0, 'NA':0,}

py_files = []

for pair in classifications:
    for pr in classifications[pair]:
        for file in classifications[pair][pr]:
            class_ = classifications[pair][pr][file]['class']
            ext = classifications[pair][pr][file]['ext']
            hunks = classifications[pair][pr][file]['hunks']
            
            if ext not in total_file_ext_classes:
                total_file_ext_classes[ext] = {}
            
            if class_ == 'OTHER EXT':
                class_= 'CC'
            if class_ == 'NOT EXISTING':
                class_ = 'NE'
                
            if ext == 'py':
                py_files.append(class_)
            try:
                total_file_ext_classes[ext][class_]+=1
            except:
                total_file_ext_classes[ext][class_] = 1

            if ext not in total_hunk_ext_classes:
                total_hunk_ext_classes[ext] = {}
            for h_list in hunks:
                for h in h_list:
                    if h not in total_hunk_ext_classes[ext]:
                        total_hunk_ext_classes[ext][h] = 1
                    total_hunk_ext_classes[ext][h] += 1

In [ ]:
total_file_ext_classes

In [ ]:
def get_hist_data(file, total_file_ext_classes):
    fec = total_file_ext_classes[file]
    labels = ['MO', 'ED', 'SP', 'NA', 'NE', 'CC', 'ERROR']
    hist = []
    for i in labels:
        try:
            hist.append(fec[i])
        except:
            hist.append(0)

    return hist

def plot_hist(file1, file2, file3, total_file_ext_classes):
    names = ['MO', 'ED', 'SP', 'NI', 'NE', 'CC', 'ERROR']
    
    if len(file3)==0 and len(file2)==0:
        fig, axs = plt.subplots(1, 1, figsize=(20, 10), sharey=True)
        axs.grid()
        axs.set_xlabel(file1)
        axs.bar(names, get_hist_data(file1, total_file_ext_classes))
    elif len(file3)==0:
        fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data(file1, total_file_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data(file2, total_file_ext_classes))
    else:    
        fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data(file1, total_file_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data(file2, total_file_ext_classes))
        axs[2].grid()
        axs[2].set_xlabel(file3)
        axs[2].bar(names, get_hist_data(file3, total_file_ext_classes))    
               
plot_hist('js', 'py', 'c', total_file_ext_classes)
plot_hist('rb', 'scala', 'cpp', total_file_ext_classes)
plot_hist('gradle', 'yml', 'ipynb', total_file_ext_classes)
plot_hist('xml', 'h', 'java', total_file_ext_classes)
plot_hist('sh', 'yaml', '', total_file_ext_classes)
# plot_hist('kt', '', '', total_file_ext_classes)
# plot_hist('js', '', '', total_file_ext_classes)


In [ ]:
def get_hist_data_hunk(ext, total_hunk_ext_classes):
    hec = total_hunk_ext_classes[ext]
    labels = ['MO', 'ED', 'SP', 'NA']
    hist = []
    for i in labels:
        try:
            hist.append(hec[i])
        except:
            hist.append(0)

    return hist

def plot_hist(file1, file2, file3, total_hunk_ext_classes):
    names = ['MO', 'ED', 'SP', 'NI']
    
    if len(file3)==0 and len(file2)==0:
        fig, axs = plt.subplots(1, 1, figsize=(20, 10), sharey=True)
        axs.grid()
        axs.set_xlabel(file1)
        axs.bar(names, get_hist_data_hunk(file1, total_hunk_ext_classes))
    elif len(file3)==0:
        fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data_hunk(file1, total_hunk_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data_hunk(file2, total_hunk_ext_classes))
    else:    
        fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
        axs[0].grid()
        axs[0].set_xlabel(file1)
        axs[0].bar(names, get_hist_data_hunk(file1, total_hunk_ext_classes))
        axs[1].grid()
        axs[1].set_xlabel(file2)
        axs[1].bar(names, get_hist_data_hunk(file2, total_hunk_ext_classes))
        axs[2].grid()
        axs[2].set_xlabel(file3)
        axs[2].bar(names, get_hist_data_hunk(file3, total_hunk_ext_classes))    
               
plot_hist('js', 'py', 'c', total_hunk_ext_classes)
plot_hist('rb', 'scala', 'cpp', total_hunk_ext_classes)
plot_hist('gradle', 'yml', 'ipynb', total_hunk_ext_classes)
plot_hist('xml', 'h', 'java', total_hunk_ext_classes)
plot_hist('sh', 'yaml', '', total_hunk_ext_classes)


In [ ]:
# MO ED SP NA NE
list_mo=[]
list_ed=[]
list_sp=[]
list_na=[]
list_ne=[]
list_lang=[]

for lang in total_file_ext_classes:
    if lang in ['c', 'h', 'java', 'cpp', 'py', 'pl', 'json', 'rb', 'scala', 'sh', \
                'php', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        
        list_lang.append(lang)
        if 'MO' in total_file_ext_classes[lang]:
            list_mo.append(total_file_ext_classes[lang]['MO'])
        else:
            list_mo.append(0)
            
        if 'ED' in total_file_ext_classes[lang]:    
            list_ed.append(total_file_ext_classes[lang]['ED'])
        else:
            list_ed.append(0)
            
        if 'SP' in total_file_ext_classes[lang]:
            list_sp.append(total_file_ext_classes[lang]['SP'])
        else:
            list_sp.append(0)
            
        if 'NA' in total_file_ext_classes[lang]:
            list_na.append(total_file_ext_classes[lang]['NA'])
        else:
            list_na.append(0)
            
        if 'NE' in total_file_ext_classes[lang]:
            list_ne.append(total_file_ext_classes[lang]['NE'])
        else:
            list_ne.append(0)
            
lang_pd = pd.DataFrame()

lang_pd['Language'] = list_lang

lang_pd['MO'] = list_mo
lang_pd['ED'] = list_ed
lang_pd['SP'] = list_sp
lang_pd['NA'] = list_na
lang_pd['NE'] = list_ne

lang_pd_mo = lang_pd[['Language', 'MO']]
lang_pd_mo.sort_values(by='MO', inplace=True)

lang_pd_ed = lang_pd[['Language', 'ED']]
lang_pd_ed.sort_values(by='ED', inplace=True)

lang_pd_sp = lang_pd[['Language', 'SP']]
lang_pd_sp.sort_values(by='SP', inplace=True)

fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)
fig.suptitle('Interesting class distribution', fontsize='30')
fig.supxlabel('Languages', fontsize='20')
# fig.supylabel('File Count', fontsize='20')

axs[0].yaxis.grid()
axs[0].set_title('MO File Distribution', fontsize='30')
axs[0].set_xticklabels(labels = lang_pd_mo[lang_pd_mo['MO']!=0]['Language'], rotation=45, ha='right', fontsize='20')
# axs[0].set_xlabel('Languages',fontsize='20')
axs[0].set_ylabel('File count',fontsize='30')
axs[0].set_yscale('log')

axs[1].yaxis.grid()
axs[1].set_title('ED File Distribution', fontsize='30')
axs[1].set_xticklabels(labels = lang_pd_ed[lang_pd_ed['ED']!=0]['Language'], rotation=45, ha='right', fontsize='20')
# axs[1].set_xlabel('Languages',fontsize='20')
# axs[1].set_ylabel('File count',fontsize='20')
axs[1].set_yscale('log')

axs[2].yaxis.grid()
axs[2].set_title('SP File Distribution', fontsize='30')
axs[2].set_xticklabels(labels = lang_pd_sp[lang_pd_sp['SP']!=0]['Language'], rotation=45, ha='right', fontsize='20')   
# axs[2].set_xlabel('Languages',fontsize='20')
# axs[2].set_ylabel('File count',fontsize='20')
axs[2].set_yscale('log')

colors_MO=[]
for i in lang_pd_mo[lang_pd_mo['MO']!=0]['Language']:
    if i in ['cpp','json', 'scala', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        colors_MO.append('turquoise')
    else:
        colors_MO.append('b')
        
colors_ED=[]
for i in lang_pd_ed[lang_pd_ed['ED']!=0]['Language']:
    if i in ['cpp','json', 'scala', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        colors_ED.append('turquoise')
    else:
        colors_ED.append('b')
        
colors_SP=[]
for i in lang_pd_sp[lang_pd_sp['SP']!=0]['Language']:
    if i in ['cpp','json', 'scala', 'gradle', 'gemfile', 'xml', 'kt', 'yml', 'yaml', 'js', 'ipynb', 'txt']:
        colors_SP.append('turquoise')
    else:
        colors_SP.append('b')
        
axs[0].bar(lang_pd_mo[lang_pd_mo['MO']!=0]['Language'], lang_pd_mo[lang_pd_mo['MO']!=0]['MO'], color=colors_MO)

axs[1].bar(lang_pd_ed[lang_pd_ed['ED']!=0]['Language'], lang_pd_ed[lang_pd_ed['ED']!=0]['ED'], color=colors_ED)

axs[2].bar(lang_pd_sp[lang_pd_sp['SP']!=0]['Language'], lang_pd_sp[lang_pd_sp['SP']!=0]['SP'], color=colors_SP)


plt.savefig("apacheKakfaGain.pdf", format="pdf", bbox_inches="tight")

In [ ]:
lang_pd

In [ ]:
with open("sorted_dataset_opt.pkl", "rb") as f:
    dataset_dict = pickle.load(f)
    
    total_prs_opt = 0
    for i in dataset_dict:
        total_prs_opt += dataset_dict[i]['pr_no']

# Get list of prs executed

In [ ]:
pr_executed={}

directory = 'Repos_results'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in pr_executed:
                pr_executed[pair_id] = []
            
            for i in results_file:
                pr_executed[pair_id].append(i)
    except EOFError:
        continue

In [ ]:
pr_executed

In [ ]:
pr_to_execute={}

directory = 'Repos_prs_opt'
for file in os.listdir(directory):
    try:
        with open(f'{directory}/{file}', 'rb') as f:
            results_file = pickle.load(f)[0]

            pair_id = file.split('_')[0]

            if pair_id not in pr_to_execute:
                pr_to_execute[pair_id] = []
            
            for i in results_file:
                if i not in pr_executed[pair_id]:
                    pr_to_execute[pair_id].append(i)
    except EOFError:
        continue

In [ ]:
pr_to_execute

# Plotting for the optimization patches

In [ ]:
import seaborn as sns

In [ ]:
data = pd.read_csv('results_opt.csv')

In [ ]:
class_list = []
count_list = []
variant1_list = []
variant2_list = []
mainlineToVariant_list = []

def appendData(row, class_, class_total):
    class_list.append(class_)
    count_list.append(row[class_total])
    variant1_list.append(row['variant1'])
    variant2_list.append(row['variant2'])
    if row['mainlineToVariant']:
        mainlineToVariant_list.append('upstream')
    else:
        mainlineToVariant_list.append('fork')
    
for idx, row in data.iterrows():
    appendData(row, 'MO', 'total_MO')
    appendData(row, 'ED', 'total_ED')
    appendData(row, 'SP', 'total_SP')
    appendData(row, 'NI', 'total_NA')
    appendData(row, 'CC', 'total_CC')
    appendData(row, 'NE', 'total_NE')
    
data_boxplot = pd.DataFrame()

data_boxplot['class'] = class_list
data_boxplot['count'] = count_list
data_boxplot['variant1'] = variant1_list
data_boxplot['variant2'] = variant2_list
data_boxplot['source'] =mainlineToVariant_list

In [ ]:
data_boxplot.to_csv('boxPlot.csv')

In [ ]:

sns.set_theme(style="whitegrid", font_scale=1.5)

g = sns.boxplot(x="class", y="count", hue="source", data=data_boxplot, palette="bright")

plt.xlabel('Classifications')
plt.ylabel('Number of patches')
plt.grid(True)
plt.legend(loc='upper left', title='source')
g.figure.set_size_inches(9,6)
g.set_yscale("log")

ax = plt.gca()
ax.grid(which='major', axis='y', linestyle='-')
ax.grid(which='major', axis='x', linestyle='')

gg = g.get_figure()
gg.savefig("boxplotOpt.pdf")

# Violin plot for the optmization patches

In [ ]:
directory = 'Repos_results'

mainline_list = []
variant_list = []
pr_list = []
filename_list = []
operation_list = []
file_class_list = []
interesting_list = []

for file in os.listdir(directory):
    with open(f'{directory}/{file}', 'rb') as f:
        results_file = pickle.load(f)[0]
        
        
        pair_id = file.split('_')[0]
        
        for i in results_file:
            for file in results_file[i]:
                mainline_list.append(dataset_dict[int(pair_id)]['source'])
                variant_list.append(dataset_dict[int(pair_id)]['destination'])
                pr_list.append(i)
                
                file_class = results_file[i][file]['result']['patchClass']
                
                filename_list.append(file)
                file_class_list.append(file_class)
                if file_class in ['MO', 'ED', 'NA', 'SP']:
                    operation_list.append(results_file[i][file]['result']['type'])
                    interesting_list.append(1)
                else:
                    operation_list.append('None')
                    interesting_list.append(0)

In [ ]:
data_violinplot = pd.DataFrame()

data_violinplot['Mainline'] = mainline_list
data_violinplot['Fork'] = variant_list
data_violinplot['Pr nr'] = pr_list
data_violinplot['Filename'] = filename_list
data_violinplot['Operation'] =operation_list
data_violinplot['File classification'] = file_class_list
data_violinplot['Interesting'] = interesting_list
data_violinplot.to_csv('Details Fig.12.csv')

In [ ]:
#Proportion of interesting files.
directory = 'Repos_results'

repo_list = []
pr_list = []
patch_list = []
interesting_list = []
total_list = []
interesting_ratio = []


for file in os.listdir(directory):
    with open(f'{directory}/{file}', 'rb') as f:
        results_file = pickle.load(f)[0]

        pr_class = totals.final_class(results_file)
        pair_id = file.split('_')[0]
        
        for i in results_file:
            patch_class = pr_class[i]['class']

            interesting_file = pr_class[i]['totals']['total_MO'] + pr_class[i]['totals']['total_ED'] + pr_class[i]['totals']['total_SP']
            total_files = pr_class[i]['totals']['total_E']
            
            if patch_class in ['MO', 'ED', 'SP']:
                repo_list.append(pair_id)
                pr_list.append(i)
                patch_list.append(patch_class)
                interesting_list.append(interesting_file)
                total_list.append(total_files)
                interesting_ratio.append(interesting_file/total_files)

In [ ]:
data_violinplot = pd.DataFrame()

data_violinplot['repo'] = repo_list
data_violinplot['pr'] = pr_list
data_violinplot['patchClass'] = patch_list
data_violinplot['interstingFilesCount'] = interesting_list
data_violinplot['totalFiles'] =total_list
data_violinplot['interestingRatio'] = interesting_ratio

In [ ]:
data_violinplot.to_csv('violionPlot.csv')

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.5)

g = sns.violinplot(x="patchClass", y="interestingRatio",
                    data=data_violinplot, palette="bright",
                    scale="count", inner = 'quartile')
sns.set(font_scale = 1.5)

plt.xlabel('Classifications')
plt.ylabel('Proportion of interesting\n files in a patch')
# plt.grid(True,  linestyle = '--')
# plt.legend(loc='upper right', title='source')
g.figure.set_size_inches(11.5,6)
# g.set_yscale("log")

ax = plt.gca()
ax.grid(which='major', axis='y', linestyle='--')
ax.grid(which='major', axis='x', linestyle='')

gg = g.get_figure()
gg.savefig("violinPlotOpt.pdf")



In [ ]:
tips

# Get the interesting optimization patches and write to file

In [ ]:
token_list = list()
token_file = '/mnt/c/Users/User1/Desktop/tokens.txt'

with open(token_file) as f:
    lines = f.readlines()
    for line in lines[0].split(','):
        token_list.append(line)

In [ ]:
directory = 'Repos_results'
import Methods.commitLoader as commitLoader
actual_pair_list = []
mainline_list = []
variant_list = []
mainline_to_variant_list = []
pr_list = []
pr_title_list = []
patch_class_list = []
ct =0

for file in os.listdir(directory):
    with open(f'{directory}/{file}', 'rb') as f:
        results_file = pickle.load(f)[0]
        
        pair_id = file.split('_')[0]
        actual_pair_id = mapping[int(pair_id)]['repo_id']
        mainline_to_variant = mapping[int(pair_id)]['mainline-variant']
        
        pr_class = totals.final_class(results_file)
        
        for i in results_file:
            actual_pair_list.append(actual_pair_id)
            mainline_list.append(dataset_dict[int(pair_id)]['source'])
            variant_list.append(dataset_dict[int(pair_id)]['destination'])
            mainline_to_variant_list.append(mainline_to_variant)
            pr_list.append(i)
            try:
                url = f'https://api.github.com/repos/' + dataset_dict[int(pair_id)]['source'] +f'/pulls/{i}'
                if ct == len(token_list):
                    ct = 0
                response = commitLoader.apiRequest(url, token_list[ct])
                ct += 1

                pr_title_list.append(response['title'])
            except:
                pr_title_list.append('')
            patch_class_list.append(pr_class[i]['class'])

In [ ]:
data_interesting= pd.DataFrame()

data_interesting['Actual Pair'] = actual_pair_list
data_interesting['Variant 1 (Source)'] = mainline_list
data_interesting['Variant 2 (Target)'] = variant_list
data_interesting['Mainline To Variant'] = mainline_to_variant_list
data_interesting['PR'] = pr_list
data_interesting['PR Title'] = pr_title_list
data_interesting['Patch classification (tool)'] = patch_class_list

data_interesting.to_csv('Interesting Optimization Patches.csv')

# Get the mapping for the optimization patches

In [ ]:
"""
    Plotting the distribution of classifications
"""
data = []

total_all = 0
total_mo_all = 0
total_ed_all = 0
total_sp_all = 0
total_cc_all = 0
total_ne_all = 0
total_error_all = 0
total_na_all = 0

all_results = []

repos = 342
for repo_file in totals_:
    try:
        variant1 = dataset_dict[int(repo_file)]['source']
        variant2 = dataset_dict[int(repo_file)]['destination']
        variant1_prs = dataset_dict[int(repo_file)]['pr'].split('/')
        
        mainlineToVariant = mapping[int(repo_file)]['mainline-variant']
        all_totals= totals_[repo_file]['pr_class']

        actua
        total_NA = 0
        total_ED = 0
        total_MO = 0
        total_CC = 0
        total_SP= 0
        total_NE = 0
        total_ERROR = 0

        for pr in all_totals:
            verdict = all_totals[pr]['class']
            if verdict == 'ED':
                total_ED += 1
            elif verdict =='MO':
                total_MO += 1
            elif verdict == 'SP':
                total_SP += 1
            elif verdict == 'NA':
                total_NA += 1
            elif verdict == 'CC':
                total_CC += 1
            elif verdict =='NE':
                total_NE += 1
            elif verdict == 'ERROR':
                total_ERROR += 1
                
            total_mid = total_MO+ total_ED + total_SP
            total_all += total_mid
            
        total_total =len(variant1_prs)
        
        total_mo_all += total_MO
        total_ed_all += total_ED
        total_sp_all += total_SP
        total_na_all += total_NA

        total_cc_all += total_CC
        total_ne_all += total_NE
        total_error_all += total_ERROR

        totals_list = [total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR]

#         analysis.all_class_bar(totals_list, repo_file, variant1, variant2, False)
#         analysis.all_class_pie(totals_list, repo_file, variant1, variant2, False)

        data.append([variant1, variant2, mainlineToVariant, repo_file, total_MO, total_ED, total_SP, total_CC, total_NE, total_NA, total_ERROR])

    except Exception as e:
        print(variant1)
        print("Exception: ", e, '\n')

# header = ["variant1", "variant2","mainlineToVariant", "repo_file", "total_MO", "total_ED", "total_SP", "total_CC", "total_NE", "total_NA"," total_ERROR"]
# with open("results_opt_29_05_22.csv", "w") as f:
#     writer = csv.writer(f)
#     # write the header
#     writer.writerow(header)

#     # write multiple rows
#     writer.writerows(all_results)